In [1]:
import config
import pymongo
import numpy as np
import pandas as pd
import json
import re

from pandas_profiling import ProfileReport
from ipywidgets import interact, Dropdown

In [2]:
myclient = pymongo.MongoClient(str(config.MONGO_URI))
stagingDb = myclient[str(config.DB_STAGING)]
analyseDb = myclient[str(config.DB_ANALYSE)]
stagingCol = stagingDb[config.COLL_PLAATJES]
stagingOud = stagingDb[config.COLL_STAGING_OUD]
stagingNieuw = stagingDb[config.COLL_STAGING_NIEUW]
analyseCol = analyseDb[config.COLL_ANALYSE]
analyseColClean = analyseDb[config.COLL_ANALYSE_CLEAN]
metaCollection = stagingDb['Kolominformatie']

In [3]:
grp_aggr = [{'$group': {'_id': {'project': "$brondata.project", 'soort': "$soort"},'teller': {"$sum": 1}}}
           ,{'$replaceRoot': {'newRoot': {'project': "$_id.project", 'soort': "$_id.soort", 'teller': { '$ifNull': ["$teller", 0]}}}}]

df_artefact = pd.DataFrame(list(analyseColClean.find({"soort": "Artefact"})))
df_artefact.head(5)

""


In [4]:
df_aardewerk = pd.DataFrame(list(analyseCol.find({"soort": "Aardewerk"})))
df_aardewerk.head(5)

""


In [5]:
df_artefact = df_artefact.merge(df_aardewerk, how='right', on=['key'])
df_artefact[df_artefact._id_x.notnull()]

KeyError: 'key'